In [1]:
import requests
import time
import pandas as pd
import numpy as np

In [2]:
api_key = "RGAPI-5e7bed4f-d23a-4574-a347-4bfb3a738849"
region = "europe"

On prend toutes les parties de chaque joueur entre le 1er Jan 2022 et le 31 Oct 2022.

On commence définit d'abord les variables utiles.

In [3]:
account_IDs = pd.read_csv("accountPUUID.csv")
account_IDs_list = account_IDs["AccountPUUID"]

#La liste des dates : date exacte des débuts et fins de mois en format "Timestamp"
dates = np.array([["Janvier", [1640995200,1643673599]],["Février", [1643673600,1646092799]],["Mars", [1646092800,1648771199]],
         ["Avril", [1648771200,1651363199]],["Mai", [1651363200,1654041599]],["Juin", [1654041600,1656633599]],
         ["Juillet", [1656633600,1659311999]],["Aout", [1659312000,1661990399]],["Septembre", [1661990400,1664582399]],
         ["Octobre", [1664582400,1667260799]]], dtype=object)

#On définit en amont le DataFrame de travail et le DataFrame final
df_vide = pd.DataFrame(columns=[i[0] for i in dates])
df = df_vide.copy()

Le programme qui nous permet de récupérer la liste des parties. (10 requêtes par puuid)

In [ ]:
def match_ID_puller(puuid,startTime,endTime):
    url_match_pull = "https://{}.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?startTime={}&endTime={}&queue=420" \
                     "&type=ranked&start=0&count=100&api_key={}".format(region, puuid, startTime, endTime, api_key)
    match_history = requests.get(url_match_pull).json()
    matchID_list += match_history

Ici, on a la boucle qui récupère tous les ID de parties et qui les sauvegardent dans un DataFrame. Globalement, pour chaque mois dans "dates", on va créer une liste ("matchID_list") et un dataFrame ("df_travail") temporaires. Dans cette liste temporaire, on va stocker pour chaque joueur toutes leurs parties jouées ce mois-ci, puis on va convertir cette liste en DataFrame temporaire ("df_travail") (dans ce DataFrame, on aura donc uniquement une seule colonne qui va contenir des ID de parties, colonne qui correspond au "mois" dans notre boucle principale). On combine ensuite le dataFrame temporaire ("df_travail") avec le DataFrame final définit plus tôt ("df") pour remplir les mois du DataFrame final un par un.

In [6]:
for mois in dates:
    
    matchID_list = []
    df_travail = df_vide.copy()
    
    for puuid in account_IDs_list:
        time.sleep(1.5)
        match_ID_puller(puuid, startTime=mois[1][0], endTime=mois[1][1])
    
    df_travail[mois[0]] = matchID_list
    df = df.combine_first(df_travail)

             Janvier Février Mars Avril  Mai Juin Juillet Aout Septembre  \
0    EUW1_5697047365     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
1    EUW1_5691273837     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
2    EUW1_5691127938     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
3    EUW1_5691060105     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
4    EUW1_5691083065     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
..               ...     ...  ...   ...  ...  ...     ...  ...       ...   
458  EUW1_5640315791     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
459  EUW1_5640295816     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
460  EUW1_5640301013     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
461  EUW1_5640180073     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   
462  EUW1_5640176466     NaN  NaN   NaN  NaN  NaN     NaN  NaN       NaN   

    Octobre  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  
.. 

In [ ]:
df.to_csv('MatchId.csv', mode='x')